# RFP: Betting on the Bachelor

## Project Overview
You are invited to submit a proposal that answers the following question:

### Who will win season 29 of the Bachelor?

*All proposals must be submitted by **1/15/25 at 11:59 PM**.*

## Required Proposal Components

### 1. Data Description
In the code cell below, read in the data you plan on using to train and test your model. Call `info()` once you have read the data into a dataframe. Consider using some or all of the following sources:
- [Scrape Fandom Wikis](https://bachelor-nation.fandom.com/wiki/The_Bachelor) or [the official Bachelor website]('https://bachelornation.com/shows/the-bachelor/')
- [Ask ChatGPT to genereate it](https://chatgpt.com/)
- [Read in csv files like this](https://www.kaggle.com/datasets/brianbgonz/the-bachelor-contestants?select=contestants.csv)

*Note, a level 5 dataset contains at least 1000 rows of non-null data. A level 4 contains at least 500 rows of non-null data.*

In [11]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup
pastsets = {'Name':[], 
           'Age': [],
            'Home town':[],
           'Occupation': [],
           'Outcome': [],
           'Place': []}
current = {'Name':[], 
           'Age': [],
           'Home town':[],
           'Occupation': [],
           'Outcome': [],
          'Place': []}
pain=[]
doublepain=[]
for i in range(1, 28):
    url=f'https://en.wikipedia.org/wiki/The_Bachelor_(American_TV_series)_season_{i}'
    response=requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for li in soup.find_all('li'):
        li.decompose()
    for sup_tag in soup.find_all('sup', class_='reference'):
        sup_tag.decompose()
    for p in soup.find_all('p'):
        p.decompose()
    try:
        
        table = soup.find('table', {'class': 'wikitable sortable'})
        if table:
            rows = table.find_all('tr')[1:]

            for tr in rows:
                cells = tr.find_all('td')
                
                if cells:  
                    name = cells[0].text.strip() if len(cells) > 0 else "N/A"
                    age = cells[1].text.strip() if len(cells) > 1 else "N/A"
                    hometown = cells[2].text.strip() if len(cells) > 2 else "N/A"
                    occupation = cells[3].text.strip() if len(cells) > 3 else "N/A"
                    outcome = cells[4].text.strip() if len(cells) > 4 else "loser"
                    place = cells[5].text.strip() if len(cells) > 5 else "N/A"
                    if name == 'N/A':
                        continue
                    else:
                        pastsets['Name'].append(name)
                        pastsets['Age'].append(age)
                        pastsets['Home town'].append(hometown)
                        pastsets['Occupation'].append(occupation)
                        pastsets['Outcome'].append(outcome)
                        pastsets['Place'].append(place)
            time.sleep(1)
        else:
            print("no table")
    except:
        pain.append(url)
        print(pain)
    
df = pd.DataFrame(pastsets)
df.info()
print("pains:", pain)
df.head()
# Don't forget to call info()!

no table
no table
no table
no table
no table
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 613 entries, 0 to 612
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        613 non-null    object
 1   Age         613 non-null    object
 2   Home town   613 non-null    object
 3   Occupation  613 non-null    object
 4   Outcome     613 non-null    object
 5   Place       613 non-null    object
dtypes: object(6)
memory usage: 28.9+ KB
pains: []


,Name,Age,Home town,Occupation,Outcome,Place
0,Amanda Marsh,23,"Chanute, Kansas",Event Planner,Winner,N/A
1,Trista Rehn,29,"St. Louis, Missouri",Miami Heat Dancer,Runner-up,N/A
2,Shannon Oliver,24,"Dallas, Texas",Financial Management Consultant,Week 5,N/A
3,Kimberly Karels,24,"Tempe, Arizona",Nanny,Week 4,N/A
4,Cathy Grimes,22,"Terre Haute, Indiana",Graduate Student,Week 3,N/A


### 2. Training Your Model
In the cell seen below, write the code you need to train a linear regression model. Make sure you display the equation of the plane that best fits your chosen data at the end of your program. 

*Note, level 5 work trains a model using only the standard Python library and Pandas. A level 5 model is trained with at least two features, where one of the features begins as a categorical value (e.g. occupation, hometown, etc.). A level 4 uses external libraries like scikit or numpy.*

In [17]:
print(df['Occupation'].unique())
occupationnumber= {'occupation':[], 'number':[]}
for i, occupation in enumerate(df['Occupation'].unique()):
    occupationnumber['occupation'].append(occupation)
    occupationnumber['number'].append(i)
occupationnumber=pd.DataFrame(occupationnumber)
occupationnumber.head()

['Event Planner' 'Miami Heat Dancer' 'Financial Management Consultant'
 'Nanny' 'Graduate Student' 'Attorney' 'Actress'
 'Commercial Real Estate Agent' 'Special Ed. Teacher'
 'Production Coordinator' 'Hooters Waitress' 'Power Tool Sales Rep.'
 'Photographer' 'Business Development Director' 'Neuropsychologist'
 'Doctor' 'Bar Manager' 'Retail Manager' 'Advertising Executive'
 'Insurance Representative' '6th Grade Teacher' 'Technology Specialist'
 'School Psychologist' 'College Student' 'Executive Recruiter'
 'Registered Nurse' 'Flight Attendant' 'Assistant Financial Advisor'
 'Marriage Therapy Trainee' 'Airline Supervisor' 'Graphic Artist'
 'Radio Sales' 'Publications Quality Control' 'Strategic Planning Analyst'
 'Psychologist' '3rd Grade Teacher' 'Radiological Technologist'
 'Interior Designer' 'Paralegal' 'Former NBA Cheerleader'
 '1st Grade Teacher' 'Communications Specialist' 'Student'
 'General Contractor' 'Architect Designer' 'Model'
 'Pharmaceutical Salesperson' 'Prosthetic Techn

,occupation,number
0,Event Planner,0
1,Miami Heat Dancer,1
2,Financial Management Consultant,2
3,Nanny,3
4,Graduate Student,4


In [22]:
df['occnum']=None
for i, player in df.iterrows():
    occupation = player['Occupation']
    assignment = occupationnumber[occupationnumber['occupation'] == occupation]
    df.loc[i, 'occnum']=assignment['number'].values[0]
df.head()

,Name,Age,Home town,Occupation,Outcome,Place,occnum
0,Amanda Marsh,23,"Chanute, Kansas",Event Planner,Winner,N/A,0
1,Trista Rehn,29,"St. Louis, Missouri",Miami Heat Dancer,Runner-up,N/A,1
2,Shannon Oliver,24,"Dallas, Texas",Financial Management Consultant,Week 5,N/A,2
3,Kimberly Karels,24,"Tempe, Arizona",Nanny,Week 4,N/A,3
4,Cathy Grimes,22,"Terre Haute, Indiana",Graduate Student,Week 3,N/A,4


In [23]:
# Train model here.
# Don't forget to display the equation of the plane that best fits your data!
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt





    
x=df[['Age', 'occnum']]
y=df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


model = LinearRegression()

model.fit(X_train, y_train)
print("slope:", model.coef_[0])
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print("Model coefficients:", model.coef_)
print("Model intercept:", model.intercept_)
print("Mean Squared Error:", mse)
print("Predictions:", y_pred)

ValueError: could not convert string to float: '(Returned to competition)'

### 3. Testing Your Model
In the cell seen below, write the code you need to test your linear regression model. 

*Note, a model is considered a level 5 if it achieves at least 60% prediction accuracy or achieves an RMSE of 2 weeks or less.*

In [2]:
# Test model here.

### 4. Final Answer

In the first cell seen below, state the name of your predicted winner. 
In the second cell seen below, justify your prediction using an evaluation technique like RMSE or percent accuracy.

#### State the name of your predicted winner here.

#### Justify your prediction here.